In [1]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [0]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [0]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [0]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [0]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [0]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=" ")
        else:
            print(colors.fail + '☒' + colors.close, end=" ")
        print(guess)
        print('---')


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 239us/step - loss: 1.8887 - acc: 0.3212 - val_loss: 1.8006 - val_acc: 0.3371


Q 419+7  
T 426 
☒ 13  
---
Q 91+70  
T 161 
☒ 131 
---
Q 761+21 
T 782 
☒ 131 
---
Q 33+952 
T 985 
☒ 131 
---
Q 41+62  
T 103 
☒ 33  
---
Q 815+54 
T 869 
☒ 151 
---
Q 843+50 
T 893 
☒ 131 
---
Q 892+3  
T 895 
☒ 131 
---
Q 846+24 
T 870 
☒ 131 
---
Q 77+764 
T 841 
☒ 171 
---

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 210us/step - loss: 1.7313 - acc: 0.3601 - val_loss: 1.6657 - val_acc: 0.3826


Q 204+795
T 999 
☒ 1303
---
Q 145+527
T 672 
☒ 102 
---
Q 17+410 
T 427 
☒ 221 
---
Q 726+1  
T 727 
☒ 121 
---
Q 419+9  
T 428 
☒ 101 
---
Q 908+853
T 1761
☒ 1113
---
Q 426+730
T 1156
☒ 1300
---
Q 68+560 
T 628 
☒ 602 
---
Q 77+88  
T 165 
☒ 188 
---
Q 493+51 
T 544 
☒ 401 
---

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 204us/step - loss: 1.6014 - acc: 0.4021 - val_loss: 1.5094 - val_acc: 0.4389


Q 78+238 
T 316 
☒ 308 
---
Q 740+586
T 1326
☒ 1379
---
Q 344+818
T 1162
☒ 1103
---
Q 233+430
T 663 
☒ 590 
---
Q 548+181
T 729 
☒ 800 
---
Q 254+45 
T 299 
☒ 560 
---
Q 824+85 
T 909 
☒ 800 
---
Q 1+560  
T 561 
☒ 666 
---
Q 434+144
T 578 
☒ 590 
---
Q 47+85  
T 132 
☒ 100 
---

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 205us/step - loss: 1.4172 - acc: 0.4687 - val_loss: 1.3322 - val_acc: 0.4974


Q 278+601
T 879 
☒ 900 
---
Q 860+248
T 1108
☒ 1006
---
Q 666+903
T 1569
☒ 1626
---
Q 38+755 
T 793 
☒ 729 
---
Q 890+58 
T 948 
☒ 906 
---
Q 412+28 
T 440 
☒ 467 
---
Q 535+820
T 1355
☒ 1266
---
Q 77+764 
T 841 
☒ 836 
---
Q 229+77 
T 306 
☑ 306 
---
Q 226+22 
T 248 
☒ 266 
---

--------------------------------------------------
Iteration 5
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 1.2575 - acc: 0.5317 - val_loss: 1.1881 - val_acc: 0.5585


...

Q 470+1  
T 471 
☑ 471 
---
Q 815+54 
T 869 
☑ 869 
---
Q 78+73  
T 151 
☑ 151 
---
Q 68+370 
T 438 
☑ 438 
---
Q 548+181
T 729 
☑ 729 
---
Q 99+328 
T 427 
☑ 427 
---
Q 90+557 
T 647 
☑ 647 
---
Q 6+252  
T 258 
☑ 258 
---
Q 66+102 
T 168 
☑ 168 
---
Q 434+203
T 637 
☑ 637 
---

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s 369us/step - loss: 9.0051e-05 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 0.9998


Q 182+8  
T 190 
☑ 190 
---
Q 17+999 
T 1016
☑ 1016
---
Q 65+523 
T 588 
☑ 588 
---
Q 0+88   
T 88  
☑ 88  
---
Q 237+747
T 984 
☑ 984 
---
Q 808+93 
T 901 
☑ 901 
---
Q 9+266  
T 275 
☑ 275 
---
Q 543+32 
T 575 
☑ 575 
---
Q 951+11 
T 962 
☑ 962 
---
Q 436+78 
T 514 
☑ 514 
---

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 18s 389us/step - loss: 8.2585e-05 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 0.9996


Q 738+28 
T 766 
☑ 766 
---
Q 5+795  
T 800 
☑ 800 
---
Q 601+58 
T 659 
☑ 659 
---
Q 53+105 
T 158 
☑ 158 
---
Q 103+208
T 311 
☑ 311 
---
Q 3+97   
T 100 
☑ 100 
---
Q 196+108
T 304 
☑ 304 
---
Q 55+754 
T 809 
☑ 809 
---
Q 7+222  
T 229 
☑ 229 
---
Q 58+668 
T 726 
☑ 726 
---

--------------------------------------------------
Iteration 186
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 353us/step - loss: 0.0284 - acc: 0.9925 - val_loss: 0.0032 - val_acc: 0.9992


Q 62+13  
T 75  
☑ 75  
---
Q 784+6  
T 790 
☑ 790 
---
Q 635+5  
T 640 
☑ 640 
---
Q 74+704 
T 778 
☑ 778 
---
Q 3+196  
T 199 
☑ 199 
---
Q 219+899
T 1118
☑ 1118
---
Q 274+987
T 1261
☑ 1261
---
Q 882+1  
T 883 
☑ 883 
---
Q 533+69 
T 602 
☑ 602 
---
Q 575+1  
T 576 
☑ 576 
---

--------------------------------------------------
Iteration 187
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 355us/step - loss: 7.5648e-04 - acc: 0.9999 - val_loss: 0.0013 - val_acc: 0.9998


Q 29+290 
T 319 
☑ 319 
---
Q 58+84  
T 142 
☑ 142 
---
Q 81+78  
T 159 
☑ 159 
---
Q 3+675  
T 678 
☑ 678 
---
Q 25+365 
T 390 
☑ 390 
---
Q 16+45  
T 61  
☑ 61  
---
Q 539+11 
T 550 
☑ 550 
---
Q 372+60 
T 432 
☑ 432 
---
Q 356+72 
T 428 
☑ 428 
---
Q 873+92 
T 965 
☑ 965 
---

--------------------------------------------------
Iteration 188
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
40576/45000 [==========================>...] - ETA: 1s - loss: 3.0203e-04 - acc: 1.0000